# Imports 

In [1]:
from common import getOrCreateSparkSession

In [2]:
spark = getOrCreateSparkSession()

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.2_2.12 added as a dependency
org.projectnessie.nessie-integrations#nessie-spark-extensions-3.2_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0e6ef28d-4f7f-41c9-aaf6-5c66edaed0c8;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.2_2.12;1.4.2 in central
	found org.projectnessie.nessie-integrations#nessie-spark-extensions-3.2_2.12;0.74.0 in central
:: resolution report :: resolve 119ms :: artifacts dl 5ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.2_2.12;1.4.2 from central in [default]
	org.projectnessie.nessie-integrations#nessie-spark-extensions-3.2_2.12;0.74.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| sear

# Branch out 

In [3]:
spark.sql("CREATE BRANCH dev IN nessie FROM main").toPandas()

,refType,name,hash
0,Branch,dev,1de998576faf154bf496cb3262ac45472e406eac2bb2a1...


In [4]:
spark.sql("LIST REFERENCES IN nessie").toPandas()

,refType,name,hash
0,Branch,dev,1de998576faf154bf496cb3262ac45472e406eac2bb2a1...
1,Branch,main,1de998576faf154bf496cb3262ac45472e406eac2bb2a1...


In [5]:
spark.sql("USE REFERENCE dev IN nessie").toPandas()

,refType,name,hash
0,Branch,dev,1de998576faf154bf496cb3262ac45472e406eac2bb2a1...


In [18]:
spark.sql(
"""
ALTER TABLE nessie.taxis.vendors ADD COLUMNS(
    address STRING
);
"""
).toPandas()

23/12/06 19:02:10 WARN BaseTransaction: Failed to load metadata for a committed snapshot, skipping clean-up


""


In [19]:
spark.sql(
"""
INSERT INTO nessie.taxis.vendors VALUES
    (1, "Yellow Taxi", "There Str"),
    (1010, "Taxis", "Onlinestrasse")
"""
).toPandas()

""


In [21]:
spark.sql("SELECT * FROM nessie.taxis.vendors LIMIT 5;").toPandas()

,vendor_id,title,address
0,1,Yellow Taxi,None
1,1,Yellow Taxi,There Str
2,10,We'll Get You There Compnay,None
3,1010,Taxis,Onlinestrasse
4,4252,Not Uber,None


# Merge back into `@main`

In [23]:
spark.sql("MERGE BRANCH dev INTO main IN nessie").toPandas()

,name,hash
0,main,d1ce11c92f031752c99fc04d120681a4559dc7989bea81...


In [24]:
spark.sql("USE REFERENCE main IN nessie").toPandas()

,refType,name,hash
0,Branch,main,d1ce11c92f031752c99fc04d120681a4559dc7989bea81...


In [25]:
spark.sql("SELECT * FROM nessie.taxis.vendors;").toPandas()

,vendor_id,title,address
0,1,Yellow Taxi,None
1,10,We'll Get You There Compnay,None
2,4252,Not Uber,None
3,1,Yellow Taxi,There Str
4,1010,Taxis,Onlinestrasse
